TODO: we should layout the objectives and questions first so that the students know what they're working towards. Otherwise, only I know what they'll be working towards whereas the students are just blindly completing activities with no tangible end result.


TODO:
when making an exercise where the students plot out number of power outages for different counties on a given day, the baseline they are comparing against is either:
a. the number of power outages that county, averaged across several days earlier in June (when the heatwave wasn't happening but it was still summer)
b. the number given in county_customers_2023 - which gives the total (modelled) number of customers in 2023 (its not 2016 but it gives you some idea of what the total might be).

# ORNL: Power Outages and Inequities in Energy Access for medically vulnerable populations

## Welcome!!
 

In June of 2016, a heatwave swept across the SW United States, causing severe loads on the power grid and leading to large number of power outages across several counties. 
We are going to be exploring correlations between temperature and power outages, and identifying how much populations relying on DME were affected in these regions.
This notebook will guide you through the stages of this project, and by the end we will have explored and identified correlations

Some reading resources: 
https://screeningtool.geoplatform.gov/en map of burdened communities 
Articles about the 2016 heatwave:
https://web.archive.org/web/20160622004100/https://weather.com/forecast/regional/news/dangerous-record-heat-southwest-plains
https://www.huffpost.com/entry/record-heat-wildfires-west-us_n_57678bb4e4b015db1bc9be59?section=
medicare at risk population map: https://empowerprogram.hhs.gov/empowermap
FIPS Codes - https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county

First things first, lets look at our data. We're going to be looking at our data using Excel (or google docs) just so you're familiar with what the data should look like. All the data we need is in this google drive folder: https://drive.google.com/drive/folders/1fAaBPgWWaA_9VB2iYUAWeW8biTXJQFwZ?usp=sharing

We are going to be working with three main datasets:
- 2016_HHSemPOWERMapHistoricalDataset.xlsx - this is the table of the medicare benificiaries population counts and the population counts of those who are dependent on DMEs. The population is listed for each county, and broken down for each month for the year 2016
- temperaturedata/CtyAvTemp6xy16.csv - in the temperaturedata folder, each .csv file is the average temperature of the day for each county in the US and its territories. The numbers in the file name represent the date. For example CtyAvTemp61716.csv is the data for June 17 2016. 
- eaglei_outages/eaglei_outages_2016.csv - This dataset gives the breakdown of the number of customers that were out of power in the year 2016, for each county in the US, with data for each 15 minute increment covering the entire year. 

**NOTE:** Something you'll find as you work with these datasets is that there might be some data missing here and there e.g. There might not be power outage information for some county for some given day. That's just the nature of data science sometimes, that the data you have isn't perfect. So you have to make sure that you're using the data you have and making sure you're accounting for any missing data when before you make any conclusions. 


## Familiarizing yourself with the dataset and generating charts with with Google Sheets

Let's start with just getting familiar with the datasets as they are before we jump into using Python. 

Download the above datasets onto your computer to open it in microsoft excel, or open it in google sheets directly from google drive if you don't have excel on your computer. 

To start with, lets open the CtyAvTemp6202016.csv file in Excel or in Sheets. We are going to try and generate a bar chart of the Average temperature in Farenheit for each county listed in the file. 

Looking at the column names, you'll notice some terms you might not be familiar with. `STATEFP` and `COUNTYFP` represent the 2 digit state part and and the 3 digit county part of the [FIPS Code](https://en.wikipedia.org/wiki/FIPS_county_code). A county or county equivalent in the US is usually given a 5 digit FIPS Code. You'll notice that the `GEOID` column represents the full FIPS Code is a combination of of the `STATEFP` and `COUNTYFP` (note that some `GEOID` values are 4 digits and some are 5 digits. The FIPS Code should be represented as always a 5 digit number, it's just that your spreadsheet decides to remove the zero if its the first digit. But the 5 digits are needed to properly identify a county. We'll be representing the full 5 digits as a string instead of a number later when we get into Python and Pandas). 

Another thing is the `TempK` column representing the average temperature in Kelvin. Lets create a column representing the temperature in Farenheit. The formula to convert Kelvin to Farenheit value is `F = 1.8*(K-273) + 32`. We don't want to manually calculate each Farenheit value, so lets see how to fill an entire column with the corresponding Farenheit value.

Create a a new column `TempF` and enter the below formula in the first cell in the new column and press enter.

```
=(1.8*(I2-273)+32)
```

Where `I2` is the id of the first cell in the `TempK` column.

![screenshot showing TempK column and formula entered in first cell of TempF column](images/sheetsformula.jpg)

Pressing enter will show two things, one is that the formula is replaced by the calculated farenheit value, and two is that google sheets might suggest an autofill to fill out the rest of the column with the same formula to convert K to F for each corresponding value in TempK. (If there is no autofill suggestion, click on the first cell in the TempF column with the formula, and double click the blue circle you see in the bottom left corner of the highlighted cell. This will fill out the formula for the rest of the column).


![screenshot highlighting the blue circle that will autofill the column](images/sheetsformulaexpand.jpg)


Next, lets create a bar chart. Obviously, creating a bar chart for the whole sheet would result in a very big bar chart, so we might want to select down our data to say a few states. For my example, I'm going to select down the data to the specific state FIPS codes 04 (AZ), 32 (NV), and 06 (CA) for my bar chart. You can do this by selcting Data->Filter Views->Create a New Filter View . Then go to the top of the columns and click on the three horizontal lines next to `STATEFP`. In the dropdown menu that opens, click on `Filter by values` and select 4,6,32 (or whichever other states you wanted to examine). 

![screenshot showing filter dropdown menu with state FPs selected](images/sheetsfilterviewselectstates.jpg)


Now creating the bar chart is pretty straightforward. Click on Insert->Chart and in the sidebar that opens select Column Chart from the Chart type dropdown. Then click on X-Axis in the sidebar and select GEOID from the list (if GEOID is not listed as an option then click on the 'Select a data range' button in the dropdown (looks like a square with a cross inside it) and then click on the column id (the alphabet above the GEOID column title) which will fill the range in the 'Select a data range' text box that opens.

![screenshot showing column selection for xaxis, column D (GEOID) is highlighted and the D column range is shown in the Select a data range textbox](images/sheetsbarchartselectrange.jpg)

Do any other customizations to your barchart from the sidebar to your liking. Try playing around with other charts to see if that gives you anything interesting.

### Exercise

Now that your familiar with navigating the data with google docs and making charts, lets try to explore  2016_HHSemPOWERMapHistoricalDataset.xlsx in the same way. When you open the xlsx file in google sheets, you will see it has three sheets 'State', 'County', 'Zip Code' which you can switch between with the tabs at the bottom. 
1. Try to create a bar chart of the DME users for the month of June. If you're using the 'State' sheet, select some states using the filters (or just do all states). If you're using the 'County' sheet, select some counties with the filter. 
2. Are you able to see any variation in population over the months in the year for a given state (or set of states)? Can you generate a chart to visualize that?
3. From exploring the data so far, is there any other perspective you would want to see that you can bring out by drawing charts?
